this looks at the clusters from time perm cluster stats and gets their average length across electrodes, and also bins them in a histogram to see the distribution  
also should get the average cluster length

In [2]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

group_mat = []

adjusted_sampling_freq = 1024

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_fixationCrossBase_1sec_mirror_0to1Test"
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Assuming `mat` is your array and `save_dir` is the directory where you want to save it
    mat_save_path = os.path.join(save_dir, f'{output_name}_mat.npy')

    # load the mat array
    mat = np.load(mat_save_path)

    group_mat.append(mat)

# Convert group_mat to a numpy array for calculation
group_mat_np = np.vstack(group_mat)  # Vertically stack the matrices

# Calculate the sum of each row for histogram
row_sums = group_mat_np.sum(axis=1)

row_sums_sig_electrodes = row_sums[row_sums > 0]

# Convert the sums to milliseconds
ms_per_sample = 1000 / adjusted_sampling_freq
row_sums_ms = row_sums * ms_per_sample
row_sums_sig_electrodes_ms = row_sums_sig_electrodes * ms_per_sample

# Plotting the histogram
plt.hist(row_sums_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Electrodes in bin')
plt.show()

KeyboardInterrupt: 

plot only sig electrodes

In [16]:
# Plotting the histogram
plt.hist(row_sums_sig_electrodes_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Sig Electrodes in bin')
plt.show()

KeyboardInterrupt: 

get avg and std of row values, to see the avg and std of total sig time across electrodes

In [18]:
avg_total_sig_time = row_sums_sig_electrodes_ms.mean()
std_total_sig_time = row_sums_sig_electrodes_ms.std()

print('avg total sig time: ', avg_total_sig_time)
print('stdev total sig time: ', std_total_sig_time)

avg total sig time:  423.0825892857143
stdev total sig time:  268.4597336795637


now get the avg length and std of each cluster, to get the avg and std of cluster size across electrodes